In [1]:
import tensorflow as tf
from numpy.random import RandomState

E:\Anaconda3\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
batch_size = 8
# 两个输入节点
x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
# 回归问题一般只有一个输出节点
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

# 定义了一个单层的神经网络前向传播过程，这里就是简单加权和
w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

In [3]:
# 定义预测多了 和 预测少了的成本
loss_less = 10
loss_more = 1
loss = tf.reduce_sum(tf.where(tf.greater(y, y_), (y - y_) * loss_more, (y_ - y) * loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

In [4]:
# 通过随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
print(X[0:3])

[[4.17022005e-01 7.20324493e-01]
 [1.14374817e-04 3.02332573e-01]
 [1.46755891e-01 9.23385948e-02]]


In [5]:
# 设置回归的正确值为两个输入的喝加上一个随机量。之所以要加上一个随机量是为了
# 加入不可预测的噪音，否则不同损失函数的意义不大，因为不同损失函数都会在能
# 完全预测正确的时候最低。一般来说噪音为一个均值为0的小量，所以这里的噪音设置为
# -0.05 ~ 0.05 的随机数
Y = [[x1 + x2 + (rdm.rand() / 10.0 - 0.05)] for (x1, x2) in X]
print(Y[0:3])

[[1.1448176486155424], [0.3152545672799197], [0.21765211375548024]]


In [6]:
# 训练神经网络
with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % 128
        end = (i * batch_size) % 128 + batch_size
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
            print("After %d training step(s), w1 is: " % (i))
            print(sess.run(w1), "\n")
    print("Final w1 is: \n", sess.run(w1))

After 0 training step(s), w1 is: 
[[-0.81031823]
 [ 1.4855988 ]] 

After 1000 training step(s), w1 is: 
[[0.01247112]
 [2.1385448 ]] 

After 2000 training step(s), w1 is: 
[[0.45567414]
 [2.1706066 ]] 

After 3000 training step(s), w1 is: 
[[0.69968724]
 [1.8465308 ]] 

After 4000 training step(s), w1 is: 
[[0.89886665]
 [1.2973602 ]] 

Final w1 is: 
 [[1.019347 ]
 [1.0428089]]
